In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp, find_peaks

# Simulation parameters
fs = 8000          # Sampling frequency (Hz)
duration = 1       # Duration of the signal (seconds)
t = np.linspace(0, duration, int(fs * duration), endpoint=False)  # Time vector

# Signal generation
frequency = 1000   # Frequency of the signal (Hz)
signal = chirp(t, f0=frequency, f1=frequency, t1=duration, method='linear')  # Chirp signal

# Microphone array parameters
num_mics = 4       # Number of microphones
mic_spacing = 0.1  # Spacing between microphones (meters)
sound_speed = 343  # Speed of sound in air (m/s)

# Direction of arrival (DOA) in radians
doa = np.pi / 4  # 45 degrees

# Time delays based on DOA
delays = np.arange(num_mics) * mic_spacing * np.sin(doa) / sound_speed
delays = np.round(delays * fs).astype(int)  # Convert to samples

# Simulate microphone signals
microphone_signals = np.zeros((num_mics, len(t)))
for i in range(num_mics):
    delayed_signal = np.roll(signal, delays[i])
    microphone_signals[i, :] = delayed_signal

# Beamforming
def delay_and_sum(signals, delays):
    num_mics, num_samples = signals.shape
    beamformed_signal = np.zeros(num_samples)
    for i in range(num_mics):
        beamformed_signal += np.roll(signals[i, :], -delays[i])
    return beamformed_signal / num_mics

# Apply beamforming
beamformed_signal = delay_and_sum(microphone_signals, delays)

# Visualization
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.title('Original Signal')
plt.plot(t, signal)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.title('Microphone Signals')
for i in range(num_mics):
    plt.plot(t, microphone_signals[i, :], label=f'Microphone {i+1}')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(3, 1, 3)
plt.title('Beamformed Signal')
plt.plot(t, beamformed_signal)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()


: 